## 大きい2D画像を切り取る

In [59]:
import cv2
import numpy as np
import glob

In [60]:
files = sorted(glob.glob('/media/dl-box/HD-PNFU3/20171210T-C-009TotalScanning'+'/*.tif'))

In [67]:
class CropWindow:
    def __init__(self, slide_step=100, window_shape=(512, 512), output_shape=(512, 512)):   
        self.slide_step = slide_step # スライドウィンドウを動かすピクセル数
        self.height = window_shape[0] # 切り取るウィンドウサイズ
        self.width = window_shape[1]
        self.output_shape = output_shape
        self.background_rate = 0.5 # この割合以上背景がある場合は無視する
        self.height_ex = False # 画像がぴったり切れなくて、余分にもう一度カットするか
        self.width_ex = False
        
    # 画像をウィンドウサイズに切り取って保存する    
    def crop_imgs(self, filenames):
        if type(filenames) == list:
            for filename in filenames:
                self.crop_save(filename)
        #　ファイル名が配列でない場合
        elif type(filenames) == str:
            self.crop_save(filenames)
    
    def crop_save(self, filename):
        img = cv2.imread(filename)
        crop_img = []
        # 何回スライドできるか、最後がぴったり画像が切り取れるかあまるか。
        if (img.shape[0] - self.height) % self.slide_step == 0:
            height_steps = (img.shape[0] - self.height)//self.slide_step
        else:
            height_steps = (img.shape[0] - self.height)//self.slide_step + 1
            self.heigth_ex = True
        if (img.shape[1] - self.height) % self.slide_step == 0:
            width_steps = (img.shape[1] - self.height)//self.slide_step
        else:
            width_steps = (img.shape[1] - self.height)//self.slide_step + 1
            self.width_ex = True
        for i in range(height_steps):
            # 最後のステップかつ、最後にずれがある場合(高さ方向)
            if i == height_steps-1 and self.height_ex:
                cropped_height = img[-self.height:, :, :]
            else:
                cropped_height = img[i*self.slide_step:i*self.slide_step+self.height, :, :]
            for j in range(width_steps):
                # 最後のステップかつ、最後にずれがある場合(横方向)
                if j == width_steps-1 and self.width_ex:
                    cropped = cropped_height[:, -self.width: , :]
                else:
                    cropped = cropped_height[:, j*self.slide_step:j*self.slide_step+self.width, :]
                if np.sum(cropped==0)/np.prod(cropped.shape) < self.background_rate:
                    crop_img.append(cropped)
        for i,pic in enumerate(crop_img):
            pic = cv2.resize(pic, self.output_shape)
            cv2.imwrite('./cropped_test/'+str(i)+'-'+filename.split('/')[-1],pic)

In [70]:
cropping = CropWindow(slide_step=512, window_shape=(512,512), output_shape=(512, 512))

In [71]:
cropping.crop_imgs(files[60])